In [66]:
#chrome://bookmarks/?id=10824

# https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/batch_prediction_model_monitoring.ipynb

#https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/custom_batch_prediction_feature_filter.ipynb

# !gsutil cp gs://bp_mm_public_data/churn/churn_bp_outsample.jsonl  churn_bp_outsample.jsonl

#https://cloud.google.com/vertex-ai/docs/samples/aiplatform-create-batch-prediction-job-bigquery-sample#aiplatform_create_batch_prediction_job_bigquery_sample-python
# https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/get-batch-predictions
import numpy as np
import pandas as pd
from google.cloud import bigquery

from datetime import date, timedelta, datetime # Date Functions
import time

In [67]:
import random
import string
# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))
UUID = generate_uuid()
print(UUID)

2fiyraxp


In [68]:
def load_data_bq(sql:str):
 client_bq = bigquery.Client()
 query_result=client_bq.query(sql)
 df_all=query_result.to_dataframe()
 return df_all

df=load_data_bq("SELECT * FROM `pongthorn.SMartML.new_incident`  LIMIT 5")
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         5 non-null      Int64         
 1   severity_id                5 non-null      Int64         
 2   severity_name              5 non-null      object        
 3   sla                        5 non-null      object        
 4   product_type               5 non-null      object        
 5   brand                      5 non-null      object        
 6   service_type               5 non-null      object        
 7   incident_type              5 non-null      object        
 8   open_to_close_hour         5 non-null      float64       
 9   response_to_resolved_hour  5 non-null      float64       
 10  imported_at                5 non-null      datetime64[ns]
dtypes: Int64(2), datetime64[ns](1), float64(2), object(6)
memory usage: 578.0+ 

,id,severity_id,severity_name,sla,product_type,brand,service_type,incident_type,open_to_close_hour,response_to_resolved_hour,imported_at
0,2452,2,Major,24x7 4Hrs Response Time,Server,HPE,Incident,Software,4.500000,4.0,2023-03-27 05:27:08.968019
1,2517,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Memory Failure,6.166667,6.0,2023-03-27 05:27:08.968019
2,2519,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Hard Disk Drive Failure,7.166667,7.0,2023-03-27 05:27:08.968019
3,2518,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Hard Disk Drive Failure,7.166667,7.0,2023-03-27 05:27:08.968019
4,2520,2,Major,24x7 4Hrs Resolution Time,Server,HPE,Incident,Hard Disk Drive Failure,7.166667,7.0,2023-03-27 05:27:08.968019


# Create batch prediction job

In [69]:
from google.cloud.aiplatform_v1beta1.types import (
    BatchDedicatedResources, BatchPredictionJob, GcsDestination, GcsSource,
    MachineSpec, ModelMonitoringAlertConfig, ModelMonitoringConfig,
    ModelMonitoringObjectiveConfig, ThresholdConfig)

In [71]:
PROJECT_ID="pongthorn"
REGION="asia-southeast1"
MODEL_ID="2102389377610022912"

TABLE_ID="pongthorn.SMartML.new_incident"

In [72]:
BATCH_JOB_NAME = f"tf1_incident_batch_{UUID}"
MODEL_URI =  f'projects/{PROJECT_ID}/locations/{location}/models/{MODEL_ID}'
INPUT_FORMAT = "bigquery"

INPUT_URI = f"bq://{TABLE_ID}"
OUTPUT_FORMAT = "bigquery"
OUTPUT_URI = f"bq://{PROJECT_ID}"

MACHINE_TYPE = "n1-standard-2"


In [73]:
EXCLUDED_FIELDS = ['id','severity_id','severity_name','imported_at']
INCLUDED_FIELDS= [ col  for col in df.columns.to_list() if col not in EXCLUDED_FIELDS ]
print(EXCLUDED_FIELDS)
print(INCLUDED_FIELDS)

['id', 'severity_id', 'severity_name', 'imported_at']
['sla', 'product_type', 'brand', 'service_type', 'incident_type', 'open_to_close_hour', 'response_to_resolved_hour']


# Create JSON body requests

In [74]:

import json

request_with_excluded_fields = {
    "displayName": f"{BATCH_JOB_NAME}-excluded_fields",
    "model": MODEL_URI,
    "inputConfig": {
        "instancesFormat": INPUT_FORMAT,
        "bigquerySource": {"inputUri": INPUT_URI},
    },
    "outputConfig": {
        "predictionsFormat": OUTPUT_FORMAT,
        "bigqueryDestination": {"outputUri": OUTPUT_URI},
    },
    "dedicatedResources": {
        "machineSpec": {
            "machineType": MACHINE_TYPE,
        }
    },
    "instanceConfig": {"excludedFields": EXCLUDED_FIELDS},
}

with open("request_with_excluded_fields.json", "w") as outfile:
    json.dump(request_with_excluded_fields, outfile)
     

In [75]:
request_with_included_fields = {
    "displayName": f"{BATCH_JOB_NAME}-included_fields",
    "model": MODEL_URI,
    "inputConfig": {
        "instancesFormat": INPUT_FORMAT,
        "bigquerySource": {"inputUri": INPUT_URI},
    },
    "outputConfig": {
        "predictionsFormat": OUTPUT_FORMAT,
        "bigqueryDestination": {"outputUri": OUTPUT_URI},
    },
    "dedicatedResources": {
        "machineSpec": {
            "machineType": MACHINE_TYPE,
        }
    },
    "instanceConfig": {"includedFields": INCLUDED_FIELDS},
}

with open("request_with_included_fields.json", "w") as outfile:
    json.dump(request_with_included_fields, outfile)

In [ ]:
# !gcloud auth application-default print-access-token
# ! curl \
#   -X POST \
#   -H "Authorization: Bearer $(gcloud auth application-default print-access-token)" \
#   -H "Content-Type: application/json" \
#   -d @request_with_excluded_fields.json \
#   https://{REGION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{REGION}/batchPredictionJobs

# Send the requests

In [84]:
#https://cloud.google.com/vertex-ai/docs/samples/aiplatform-create-batch-prediction-job-bigquery-sample#aiplatform_create_batch_prediction_job_bigquery_sample-python

In [81]:
parent = f"projects/{PROJECT_ID}/locations/{REGION}"
response = client.create_batch_prediction_job(
    parent=parent, batch_prediction_job=batch_prediction_job
)
print("response:", response)